'#number' in each cell represent the sequence to run the cells

In [1]:
#1
import os
import pandas as pd
import csv
from colour import Color
import matplotlib


In [32]:
#edit the path to folder and run at number 2
path='.../data-set 3'

In [33]:
#edit the exportpath as per your machine and reun at number 3
exportpath='.../out_data/out/'

In [1]:
#12
#this function takes the path of the folder as input and 
#returns the two dataframes- variant dataframe and Phenotype dataframe 
# this function gives call to several other function doing subtasks

def starthere(path):
    variants_df=pd.DataFrame()
    phenotype_df=pd.DataFrame()
    
    for folder in os.listdir(path):
        if folder.startswith('.DS_Store'):
            continue
        
        curr_path="".join((path, "/",folder,))
        files=os.listdir(curr_path)
        if '.DS_Store' in files:
            files=files.remove('.DS_Store')
        
       
        varpath="".join((curr_path,"/",files[1]))
        if varpath.endswith('.txt'):
            temp=readvar(varpath)
            processed=process(temp)
            processed.loc[:,'person_id'] = folder
            variants_df=variants_df.append(processed)
    
      
        
        phenopath="".join((curr_path,"/",files[0]))
        
        if phenopath.endswith('.txt'):
            temp=readpheno(phenopath)
            temp['person_id']=''
            temp.loc[:,'person_id'] = folder
           
            phenotype_df=phenotype_df.append(temp)
            

    
    phenotype_df=normalize(pd.DataFrame(phenotype_df))
    
    return variants_df,phenotype_df 

In [35]:
#13
variant_database,phenotype_database=starthere(path)

<ipython-input-9-04a635a261a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['genotype']=file[3]+file[4]


In [36]:
#14
variant_database

,rsid,chromosome,position,genotype,person_id
1,rs4345758,1,28663,TT,PY633MJ399
2,rs4477212,1,72017,AA,PY633MJ399
3,rs1856862,1,110905,AA,PY633MJ399
4,rs4096703,1,160593,GG,PY633MJ399
5,rs2994597,1,593369,--,PY633MJ399
...,...,...,...,...,...
677860,rs41534744,26,16129,AA,285536LN7G
677861,rs41419246,26,16145,GG,285536LN7G
677862,rs41466049,26,16162,AA,285536LN7G
677863,rs41355449,26,16327,CC,285536LN7G


In [37]:
#15
phenotype_database

,Characteristic,Variation,person_id
0,Handedness,Right-handed,PY633MJ399
1,Eye color,brown,PY633MJ399
0,ABO Rh,A-,SEBNF3PGUJ
1,Blood type,A-,SEBNF3PGUJ
2,Interest in Spirituality and Mysticism,None,SEBNF3PGUJ
...,...,...,...
4,Handedness,Right-handed,285536LN7G
5,Eye color,brown,285536LN7G
6,Lisp,No,285536LN7G
7,Sweat eating spicy food,Sometimes,285536LN7G


In [38]:
#16
export_to_csv(phenotype_database,exportpath,'phenotype_database')

In [39]:
#17
export_to_csv(variant_database,exportpath,'variant_database')

In [4]:
#4
# input= file: dataframe to export, exportpath, name- name of the csv file you want to create without .csv extension
#output= csv file will be created in the file system
def export_to_csv(file,exportpath,name):
    path=str(exportpath+name+".csv")
    file.to_csv(path,index = False)
    

In [5]:
#5
#input= variant dataset
#output= normalized variant data that has cossistent values green-blue and blue-green are entred as one value
def normalize(vari):
    color=[]
    l=[]
    seen=[]

    for name, hex in matplotlib.colors.cnames.items():
        color.append(name)
        
    for ind,var in enumerate(vari['Variation']):
        var=var.lstrip()
        var=var.rstrip()
        var=var.lower()
        if var.startswith(tuple(color)):
            l.append([ind,var])
           
    import re
    
    for i in l:
        ind=i[0]
        res = re.split(',|_|-|!| ', i[1])
        
        for word in res:
            if word not in color:
                res.remove(word)
                
        ressort=sorted(res)
    
        if res not in seen:
            seen.append(res)
        newst='-'.join(ressort)
      
        vari.iat[ind,1]=newst

    return vari

In [6]:
#6
#input= path of variant.txt file
#output= returns the data read from file in file system
# this function reads the variant.txt files
def readvar(variant):
    skip_rows = 0
    with open(variant, 'r') as f:
        for line in f:
            if line.startswith('#'):
                skip_rows += 1
            else:
                break
    
    with open(variant, newline='') as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.read(1024))
    sepe = dialect.delimiter
    if sepe!=',':
        sepe='\t'
    file=pd.read_csv(variant,comment='#',skiprows=skip_rows,sep=sepe,header=None,error_bad_lines=False,low_memory=False)
    #print('in readvar ',file[:5])
    return file
    


In [7]:
#7
#input= path of Phenotype.txt file
#output= returns the data read from file in file system
# this function reads the phenitype.txt files
def readpheno(pheno):
    file= pd.read_csv(pheno,sep='\t',error_bad_lines=False)
    file=pd.DataFrame(file)
    return file

In [8]:
#8
#input= variant.txt file with 6 columns
#output= processed and cleaned file as per the given  and standard guidelines
#this function converts genotype(coded with repsect to negative strand ) to genorype that is recorded with repsect to pisitive strand and renames the columns in consistant fashion

def process6cols(file):
    dic={'A':'T','T':'A','C':'G','G':'C','-':'-'}
    if isinstance(file[0][1], str):
        file=file[1:]
    for index, row in file.iterrows():
        code=row[5]
        if row[4]=='-':
            file.at[index,'genotype'] = dic[code[0]]+dic[code[1]]
        else:
            file.at[index,'genotype'] = code
    
    file=file.rename(columns={0: 'rsid', 2: 'chromosome',3:'position'})
    file.drop([1,4,5],axis=1,inplace=True)
    return pd.DataFrame(file)
    

In [9]:
#9
#input= variant.txt file with 5 columns
#output= processed and cleaned file as per the given  and standard guidelines
#this function merges two saperate strands in one column-genotype as required, renames the columns in consistant fashion
def process5cols(file):
    if isinstance(file[0][1], str):
        file=file[1:]
    file['genotype']=file[3]+file[4]
    file=file.rename(columns={0: 'rsid', 1: 'chromosome',2:'position'})
    file=file.drop([3,4], axis = 1)
    file['genotype']=file['genotype'].replace({'00':'--'},regex=True)
    return pd.DataFrame(file)


In [10]:
#10
#input= variant.txt file with 4 columns
#output= processed and cleaned file as per the given  and standard guidelines
#renames the columns in consistant fashion, removes the comment 
def process4cols(file):
    #print(file.columns)
    if isinstance(file[0][1], str):
        file=file[1:]
    file=file.rename(columns={0: 'rsid', 1: 'chromosome',2:'position',3:'genotype'})
    file['genotype']=file['genotype'].replace({'00':'--'},regex=True)
    return pd.DataFrame(file)


In [11]:
#11
# input= variant file
# output= returns the cleaned variant file
# this function calls different cleaning function to clean the files in different format
# it checks for the number of columns in file, since number of columns is deciding factor in file format in this case
def process(file):
    cols = len(file.axes[1])
    if cols==6:
        data6=process6cols(file)

        return data6
    elif cols==5:
        data5=process5cols(file)

        return data5
    elif cols==4:
        data4=process4cols(file)
        return data4

In [40]:
#18
# creating database with unique combination of characteristics and variants 
char_var=phenotype_database[['Characteristic ','Variation']].drop_duplicates()

In [41]:
#19
# exporting database with unique combination of characteristics and variants 
export_to_csv(char_var,exportpath,'char_var')

In [42]:
#20
# creating database with unique combination of personid and characteristics 
person_char=phenotype_database[['person_id','Characteristic ']].drop_duplicates()

In [43]:
#21
# exporting database with unique combination of personid and characteristics 
export_to_csv(person_char,exportpath,'person_char')